In [1]:
'''
Data used here is from Northern Hemisphere EASE-Grid 2.0 Weekly Snow Cover and Sea Ice Extent, Version 4

''' 

# Modules used in this program

%matplotlib qt
import yaml
yaml.warnings({'YAMLLoadWarning': False})
# the default loader is deprecated, I dont know how to change the default loader
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import random
import math
from scipy import stats
import seaborn as sns 
import pandas as pd
from mpl_toolkits.basemap import Basemap
#random.seed(1234)

In [ ]:
# Underlying map
# Drawing an Lambert Azimuthal Projection
def draw_map1():
    m = Basemap(projection='laea', resolution=None, lat_0=90,lon_0=0,width = 25*1000*720,height =25*1000*720, boundinglat=0, round=True)
    m.shadedrelief(scale=0.7)
    return m
def draw_map2():
    m = Basemap(projection='laea', resolution=None, lat_0=90,lon_0=0,width = 25*1000*720,height =25*1000*720, boundinglat=0, round=True)
    m.bluemarble(scale=0.7)
    return m
def draw_map3():
    
    m = Basemap(projection='laea', resolution=None, lat_0=90,lon_0=0,width = 25*1000*720,height =25*1000*720, boundinglat=0, round=True)
    m.drawlsmask(land_color='mediumspringgreen',ocean_color='lightskyblue', lakes=True)
    return m
def draw_map4():
    
    m = Basemap(projection='laea', resolution=None, lat_0=90,lon_0=0,width = 25*1000*720,height =25*1000*720, boundinglat=0, round=False)
    #m.drawlsmask(land_color='gray',ocean_color='white', lakes=True)
    return m
#def draw_map4():
 #   m = Basemap(projection='laea', resolution=None, lat_0=90,lon_0=0,width = 25*1000*720,height =25*1000*720, boundinglat=0, round=True)
 #   m.drawcoastlines()
    #  #m.drawmapboundary(fill_color='bisque')
    #m.fillcontinents(color='sandybrown',lake_color='bisque')
   # return m
def draw_map5():
    m = Basemap(projection='laea', resolution=None, lat_0=90,lon_0=0,width = 25*1000*720,height =25*1000*720, boundinglat=0, round=True)
    m.etopo()
    return m

In [18]:
# plotting parameters
parallels = np.arange(0,90,10.)
meridians = np.arange(0.,360.,30.)

In [3]:
# Reading the netcdf file

whole_dataset = r"C:\Users\krish\Desktop\Courses\Thesis\data\downloadtrials\snow\complete.nc"
dataset = xr.open_dataset(whole_dataset, chunks = {'time':10})#, 'row':10, 'col':10})

In [4]:
# Attributes of netcdf file

dataset

<xarray.Dataset>
Dimensions:     (col: 720, row: 720, time: 2689)
Coordinates:
  * row         (row) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * col         (col) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * time        (time) datetime64[ns] 1966-10-03 1966-10-10 ... 2018-12-24
Data variables:
    snow_cover  (time, row, col) int8 dask.array<shape=(2689, 720, 720), chunksize=(10, 720, 720)>
    lats        (row, col) float64 dask.array<shape=(720, 720), chunksize=(720, 720)>
    lons        (row, col) float64 dask.array<shape=(720, 720), chunksize=(720, 720)>

In [6]:
# Narrowing the dataset to spring values

springs = []
for year in np.arange(1979,2019):
    beginning = "%d-03-01" %(year)
    end = "%d-08-30" %(year)
    dataset_spring = dataset.sel(time=slice(beginning,end))
    
    # Dataset snowcover values resampled from Weekly data to Monthly data here
    springs.append(dataset_spring.resample(time='M').mean())
    


all_springs = xr.concat(springs, 'time', data_vars = 'minimal')

In [7]:
# Description of the processed spring snowcover data. 
all_springs

<xarray.Dataset>
Dimensions:     (col: 720, row: 720, time: 240)
Coordinates:
  * row         (row) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * col         (col) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * time        (time) datetime64[ns] 1979-03-31 1979-04-30 ... 2018-08-31
Data variables:
    snow_cover  (time, row, col) float64 dask.array<shape=(240, 720, 720), chunksize=(1, 720, 720)>
    lats        (time, row, col) float64 dask.array<shape=(240, 720, 720), chunksize=(1, 720, 720)>
    lons        (time, row, col) float64 dask.array<shape=(240, 720, 720), chunksize=(1, 720, 720)>

In [8]:
# Climatology plots

March, April, May = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)) 
June, July, August = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))
for i in np.arange(0,40):
    
    March += all_springs.snow_cover[0+i*6,:,:].values
    April += all_springs.snow_cover[1+i*6,:,:].values
    May += all_springs.snow_cover[2+i*6,:,:].values
    June += all_springs.snow_cover[3+i*6,:,:].values
    July += all_springs.snow_cover[4+i*6,:,:].values
    August += all_springs.snow_cover[5+i*6,:,:].values



In [9]:
# Calculating the Mean for each month

March = March/40
April = April/40
May = May/40
June = June/40
July = July/40
August = August/40


In [173]:
# Plotting the Climatology Maps
ticks = np.linspace(0,1,9)
color = plt.cm.get_cmap('Blues', 8)

plt.figure()
mp = draw_map4()
plt.title("Climatology map for March (1979-2018)", y=1.08)
mp.imshow(March, origin = 'lower', cmap = color)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp = draw_map4()
plt.title("Climatology map for April (1979-2018)", y=1.08)
mp.imshow(April, origin = 'lower', cmap = color)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp = draw_map4()
plt.title("Climatology map for May (1979-2018)", y=1.08)
mp.imshow(May, origin = 'lower', cmap = color)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp = draw_map4()
plt.title("Climatology map for June (1979-2018)", y=1.08)
mp.imshow(June, origin = 'lower', cmap = color)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp = draw_map4()
plt.title("Climatology map for July (1979-2018)", y=1.08)
mp.imshow(July, origin = 'lower', cmap = color)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp = draw_map4()
plt.title("Climatology map for August (1979-2018)", y=1.08)
mp.imshow(August, origin = 'lower', cmap = color)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.show()

In [88]:
# Trend map

# Execution of this block takes quite 
years = np.arange(0, 40) 
slopes_march = np.empty([720, 720])
intercepts_march = np.empty([720, 720])
slopes_april = np.empty([720, 720])
intercepts_april = np.empty([720, 720])
slopes_may = np.empty([720, 720])
intercepts_may = np.empty([720, 720])
slopes_june = np.empty([720, 720])
intercepts_june = np.empty([720, 720])
slopes_july = np.empty([720, 720])
intercepts_july = np.empty([720, 720])
slopes_august = np.empty([720, 720])
intercepts_august = np.empty([720, 720])

for row in np.arange(720):
    march_values = []
    april_values = []
    may_values = []
    june_values = []
    july_values = []
    august_values = []
    w_array_march = all_springs.snow_cover[0:240:6,row,:].values
    w_array_april = all_springs.snow_cover[1:240:6,row,:].values
    w_array_may = all_springs.snow_cover[2:240:6,row,:].values
    w_array_june = all_springs.snow_cover[3:240:6,row,:].values
    w_array_july = all_springs.snow_cover[4:240:6,row,:].values
    w_array_august = all_springs.snow_cover[5:240:6,row,:].values
    print(row)
    for col in np.arange(720):
        
        march_values.append(w_array_march[:,col])
        april_values.append(w_array_april[:,col])
        may_values.append(w_array_may[:,col])
        june_values.append(w_array_june[:,col])
        july_values.append(w_array_july[:,col])
        august_values.append(w_array_august[:,col])
        
        m, b = np.polyfit(years, march_values[col], 1)
        slopes_march[row, col] = m
        intercepts_march[row, col] = b
        
        m, b = np.polyfit(years, april_values[col], 1)
        slopes_april[row, col] = m
        intercepts_april[row, col] = b
        
        m, b = np.polyfit(years, may_values[col], 1)
        slopes_may[row, col] = m
        intercepts_may[row, col] = b
        
        m, b = np.polyfit(years, june_values[col], 1)
        slopes_june[row, col] = m
        intercepts_june[row, col] = b
        
        m, b = np.polyfit(years, july_values[col], 1)
        slopes_july[row, col] = m
        intercepts_july[row, col] = b
        
        m, b = np.polyfit(years, august_values[col], 1)
        slopes_august[row, col] = m
        intercepts_august[row, col] = b
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [126]:
print(len(w_array_march),len(march_values[1]))
print(np.amax(slopes_march), np.amin(slopes_march), np.mean(slopes_march))
print(np.amax(slopes_april), np.amin(slopes_april), np.mean(slopes_april))
print(np.amax(slopes_may), np.amin(slopes_may), np.mean(slopes_may))
print(np.amax(slopes_june), np.amin(slopes_june), np.mean(slopes_june))
print(np.amax(slopes_july), np.amin(slopes_july), np.mean(slopes_july))
print(np.amax(slopes_august), np.amin(slopes_august), np.mean(slopes_august))

40 40
0.029263602251407117 -0.02281425891181988 -0.00017149570297118508
0.02326454033771106 -0.02379455909943714 -0.0001823134178190067
0.024029080675422133 -0.02774390243902438 -0.0003134996029781342
0.018513133208255154 -0.029301125703564725 -0.000368782047662891
0.0274249530956848 -0.02827861163227014 -0.00015178149537905622
0.029301125703564725 -0.02128048780487804 -7.821688142762828e-05


In [184]:
# Plots of trend maps
ticks = np.round(np.linspace(-0.04,0.04,12),3)
color = plt.cm.get_cmap('seismic', 11)
color.set_bad(color='white')
plt.figure()
mp=draw_map4()
plt.title("Trend map for March (1979-2018)", y=1.08)
#cmap = colors.ListedColormap(['black', 'white'])
mp.imshow(slopes_march, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Trend map for April (1979-2018)", y=1.08)
mp.imshow(slopes_april, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Trend map for May (1979-2018)", y=1.08)
mp.imshow(slopes_may, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Trend map for June (1979-2018)", y=1.08)
mp.imshow(slopes_june, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Trend map for July (1979-2018)", y=1.08)
mp.imshow(slopes_july, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Trend map for August (1979-2018)", y=1.08)
mp.imshow(slopes_august, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks=ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.show()

In [ ]:
# Anomaly maps

#Variables storing values of 1st 20 years of the dataset
March_b, April_b, May_b, June_b, July_b, August_b = np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720])
#Variables storing values of last 20 years of the dataset
March_l, April_l, May_l, June_l, July_l, August_l = np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720])


for i in np.arange(0,40):
    
    if i < 20:
        March_b += all_springs.snow_cover[0+i*6,:,:].values
        April_b += all_springs.snow_cover[1+i*6,:,:].values
        May_b += all_springs.snow_cover[2+i*6,:,:].values
        June_b += all_springs.snow_cover[3+i*6,:,:].values
        July_b += all_springs.snow_cover[4+i*6,:,:].values
        August_b += all_springs.snow_cover[5+i*6,:,:].values
    else:
        March_l += all_springs.snow_cover[0+i*6,:,:].values
        April_l += all_springs.snow_cover[1+i*6,:,:].values
        May_l += all_springs.snow_cover[2+i*6,:,:].values
        June_l += all_springs.snow_cover[3+i*6,:,:].values
        July_l += all_springs.snow_cover[4+i*6,:,:].values
        August_l += all_springs.snow_cover[5+i*6,:,:].values

In [124]:
anomaly_march = (March_l - March_b)/400
anomaly_april = (April_l - April_b)/400
anomaly_may = (May_l - May_b)/400
anomaly_june = (June_l - June_b)/400
anomaly_july = (July_l - July_b)/400
anomaly_august = (August_l - August_b)/400

In [125]:
print(np.amax(anomaly_march), np.amin(anomaly_march), np.mean(anomaly_march))
print(np.amax(anomaly_april), np.amin(anomaly_april), np.mean(anomaly_april))
print(np.amax(anomaly_may), np.amin(anomaly_may), np.mean(anomaly_may))
print(np.amax(anomaly_june), np.amin(anomaly_june), np.mean(anomaly_june))
print(np.amax(anomaly_july), np.amin(anomaly_july), np.mean(anomaly_july))
print(np.amax(anomaly_august), np.amin(anomaly_august), np.mean(anomaly_august))

0.031875 -0.0275 -0.00014952136381172847
0.028625 -0.031875 -0.00020775318287037035
0.027500000000000004 -0.03375 -0.0003148439911265433
0.021625000000000002 -0.038125000000000006 -0.0003727367862654321
0.03525 -0.030625 -0.00015519097222222215
0.039625 -0.028999999999999998 -8.825834297839507e-05


In [186]:
# Plots of Anomaly maps
ticks = np.round(np.linspace(-0.04,0.04,12),3)
color = plt.cm.get_cmap('seismic', 11)
plt.figure()    
mp=draw_map4()
plt.title("Anomaly map for March (1979-2018)", y=1.08)

mp.imshow(anomaly_march, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Anomaly map for April (1979-2018)", y=1.08)
mp.imshow(anomaly_april, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Anomaly map for May (1979-2018)", y=1.08)
mp.imshow(anomaly_may, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Anomaly map for June (1979-2018)", y=1.08)
mp.imshow(anomaly_june, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Anomaly map for July (1979-2018)", y=1.08)
mp.imshow(anomaly_july, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
plt.title("Anomaly map for August (1979-2018)", y=1.08)
mp.imshow(anomaly_august, vmax = 0.04, vmin = -0.04, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.show()

In [194]:
def find_average(snowcover_data):
    """
    Returns the spatial average snowcover and spatial aggregate snowcover for a particular month
    """
    total = np.sum(snowcover_data)
    average = total/(720*720)
    #total is multiplied with 625 for spatial aggregate as the area of each grid cell is 25km*25km and divided by 10^6 to units of millions
    return average, total*625/(10**6)

In [198]:
# Trend lines


coverage_march = []
coverage_april = []
coverage_may = []
coverage_june = []
coverage_july = []
coverage_august = []

average_march = find_average(March)[0]
average_april = find_average(April)[0]
average_may = find_average(May)[0]
average_june = find_average(June)[0]
average_july = find_average(July)[0]
average_august = find_average(August)[0]

for i in np.arange(0,40):
    coverage_march.append(find_average(all_springs.snow_cover[0+i*6,:,:].values)[0])
    coverage_april.append(find_average(all_springs.snow_cover[1+i*6,:,:].values)[0])
    coverage_may.append(find_average(all_springs.snow_cover[2+i*6,:,:].values)[0])
    coverage_june.append(find_average(all_springs.snow_cover[3+i*6,:,:].values)[0])
    coverage_july.append(find_average(all_springs.snow_cover[4+i*6,:,:].values)[0])
    coverage_august.append(find_average(all_springs.snow_cover[5+i*6,:,:].values)[0]) 


In [199]:
#plot of trend lines:
years = np.arange(1979, 2019)
sns.set()
plt.figure()
plt.title("Trend of Snow cover -NH- March 1979-2018")

plt.plot(years, np.subtract(coverage_march, average_march)/average_march)
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snow cover to (1979-2018)")

plt.figure()
plt.title("Trend of Snow cover -NH- April 1979-2018")

plt.plot(years, np.subtract(coverage_april, average_april)/average_april)
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snow cover to (1979-2018)")

plt.figure()
plt.title("Trend of Snow cover -NH- May 1979-2018")

plt.plot(years, np.subtract(coverage_may, average_may)/average_may)
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snow cover to (1979-2018)")

plt.figure()
plt.title("Trend of Snow cover -NH- June 1979-2018")

plt.plot(years, np.subtract(coverage_june, average_june)/average_june)
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snow cover to (1979-2018)")

plt.figure()
plt.title("Trend of Snow cover -NH- July 1979-2018")

plt.plot(years, np.subtract(coverage_july, average_july)/average_july)
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snow cover to (1979-2018)")

plt.figure()
plt.title("Trend of Snow cover -NH- August 1979-2018")

plt.plot(years, np.subtract(coverage_august, average_august)/average_august)
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snow cover to (1979-2018)")

plt.show()

In [200]:
# TimeSeries of Snowcover for each month.
coverage_march = []
coverage_april = []
coverage_may = []
coverage_june = []
coverage_july = []
coverage_august = []

for i in np.arange(0,40):
    coverage_march.append(find_average(all_springs.snow_cover[0+i*6,:,:].values)[1])
    coverage_april.append(find_average(all_springs.snow_cover[1+i*6,:,:].values)[1])
    coverage_may.append(find_average(all_springs.snow_cover[2+i*6,:,:].values)[1])
    coverage_june.append(find_average(all_springs.snow_cover[3+i*6,:,:].values)[1])
    coverage_july.append(find_average(all_springs.snow_cover[4+i*6,:,:].values)[1])
    coverage_august.append(find_average(all_springs.snow_cover[5+i*6,:,:].values)[1])
 

In [201]:
   
# Adding monthly coverage to a dataframe
years = np.arange(1979, 2019)
df_total_coverage = pd.DataFrame(list(zip(coverage_march, coverage_april, coverage_may, coverage_june, coverage_july, coverage_august)), columns=['March','April', 'May','June','July', 'August'], index=years)

In [202]:
#plot of snowcover timeseries:

sns.set()
plt.title("Time Series of Snow cover -NH- 1979-2018")

sns.lineplot(data=df_total_coverage)

plt.xlabel("Years")
plt.ylabel("Aggregate snow cover (in million km^2)")

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

In [10]:
# Composite Maps

#Importing year categories from ERA-Interim-onlinecopy script 

%store -r july_c3_dominated 
%store -r july_c1_dominated 
%store -r july_neither_dominated
%store -r august_c3_dominated 
%store -r august_c1_dominated
%store -r august_neither_dominated

In [11]:
# Delete this cell
august_c1_dominated

array([1980, 1981, 1982, 1986, 1992, 1994, 1997, 2003, 2004, 2006, 2013,
       2016, 2017], dtype=int64)

In [12]:
# Next step is to use seggregated years (c1 dominated, c3 dominated, neither) to find average plots for each variable 
# within these particular years
# And now its monthly data
# Taking into consideration the zonal means
# And now making the climatology plots

May_jc1, June_jc1, July_jc1, August_jc1 = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))
May_jc3, June_jc3, July_jc3, August_jc3 = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))
May_jn, June_jn, July_jn, August_jn = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))

May_ac1, June_ac1, July_ac1, August_ac1 = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))
May_ac3, June_ac3, July_ac3, August_ac3 = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))
May_an, June_an, July_an, August_an = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))

for i in np.arange(0,40):
    yr = i+1979
    if yr in july_c1_dominated:
        May_jc1 += all_springs.snow_cover[2+i*6,:,:].values
        June_jc1 += all_springs.snow_cover[3+i*6,:,:].values
        July_jc1 += all_springs.snow_cover[4+i*6,:,:].values
        August_jc1 += all_springs.snow_cover[5+i*6,:,:].values
    elif yr in july_c3_dominated:
        May_jc3 += all_springs.snow_cover[2+i*6,:,:].values
        June_jc3 += all_springs.snow_cover[3+i*6,:,:].values
        July_jc3 += all_springs.snow_cover[4+i*6,:,:].values
        August_jc3 += all_springs.snow_cover[5+i*6,:,:].values
    elif yr in july_neither_dominated:
        May_jn += all_springs.snow_cover[2+i*6,:,:].values
        June_jn += all_springs.snow_cover[3+i*6,:,:].values
        July_jn += all_springs.snow_cover[4+i*6,:,:].values
        August_jn += all_springs.snow_cover[5+i*6,:,:].values
        
    if yr in august_c1_dominated:
        May_ac1 += all_springs.snow_cover[2+i*6,:,:].values
        June_ac1 += all_springs.snow_cover[3+i*6,:,:].values
        July_ac1 += all_springs.snow_cover[4+i*6,:,:].values
        August_ac1 += all_springs.snow_cover[5+i*6,:,:].values
    elif yr in august_c3_dominated:
        May_ac3 += all_springs.snow_cover[2+i*6,:,:].values
        June_ac3 += all_springs.snow_cover[3+i*6,:,:].values
        July_ac3 += all_springs.snow_cover[4+i*6,:,:].values
        August_ac3 += all_springs.snow_cover[5+i*6,:,:].values
    elif yr in august_neither_dominated:
        May_an += all_springs.snow_cover[2+i*6,:,:].values
        June_an += all_springs.snow_cover[3+i*6,:,:].values
        July_an += all_springs.snow_cover[4+i*6,:,:].values
        August_an += all_springs.snow_cover[5+i*6,:,:].values


In [13]:
# Averages for each of these months

# Calculating the Mean for each month

May_jc1, June_jc1, July_jc1, August_jc1 = May_jc1/len(july_c1_dominated), June_jc1/len(july_c1_dominated), July_jc1/len(july_c1_dominated), August_jc1/len(july_c1_dominated)
May_jc3, June_jc3, July_jc3, August_jc3 = May_jc3/len(july_c3_dominated), June_jc3/len(july_c3_dominated), July_jc3/len(july_c3_dominated), August_jc3/len(july_c3_dominated)
May_jn, June_jn, July_jn, August_jn = May_jn/len(july_neither_dominated), June_jn/len(july_neither_dominated), July_jn/len(july_neither_dominated), August_jn/len(july_neither_dominated)

May_ac1, June_ac1, July_ac1, August_ac1 = May_ac1/len(august_c1_dominated), June_ac1/len(august_c1_dominated), July_ac1/len(august_c1_dominated), August_ac1/len(august_c1_dominated)
May_ac3, June_ac3, July_ac3, August_ac3 = May_ac3/len(august_c3_dominated), June_ac3/len(august_c3_dominated), July_ac3/len(august_c3_dominated), August_ac3/len(august_c3_dominated)
May_an, June_an, July_an, August_an = May_an/len(august_neither_dominated), June_an/len(august_neither_dominated), July_an/len(august_neither_dominated), August_an/len(august_neither_dominated)

In [44]:
# Plotting the Composite Maps

# July-c1 dominated years
ticks = np.round(np.linspace(-0.25,0.25,10),3)
color = plt.cm.get_cmap('seismic', 9)

plt.figure()
plt.suptitle("Snowcover in May, June, July and August relative to average snowcover(1979-2018)(C1-July Dominated)")
mp=draw_map4()
plt.subplot(2,2,1)
plt.title("May", y=1.08)
mp.imshow(May_jc1-May, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,2)
plt.title("June", y=1.08)
mp.imshow(June_jc1-June, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,3)
plt.title("July", y=1.08)
mp.imshow(July_jc1-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,4)
plt.title("August", y=1.08)
mp.imshow(August_jc1-August, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.subplots_adjust(wspace=0.,hspace=0.4, left=0.2, right=0.8)
#plt.tight_layout()
plt.show()

In [45]:
# July-c3 dominated years
ticks = np.round(np.linspace(-0.25,0.25,10),3)
color = plt.cm.get_cmap('seismic', 9)

plt.figure()
plt.suptitle("Snowcover in May, June, July and August relative to average snowcover(1979-2018)(C3-July Dominated)")

mp=draw_map4()
plt.subplot(2,2,1)
plt.title("May", y=1.08)
mp.imshow(May_jc3-May, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,2)
plt.title("June", y=1.08)
mp.imshow(June_jc3-June, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,3)
plt.title("July", y=1.08)
mp.imshow(July_jc3-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,4)
plt.title("August", y=1.08)
mp.imshow(August_jc3-August, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.subplots_adjust(wspace=0.,hspace=0.4, left=0.2, right=0.8)
plt.show()

In [46]:
# July-neither dominated years
ticks = np.round(np.linspace(-0.25,0.25,10),3)
color = plt.cm.get_cmap('seismic', 9)

plt.figure()
plt.suptitle("Snowcover in May, June, July and August relative to average snowcover(1979-2018)(Neither-July Dominated)")

mp=draw_map4()
plt.subplot(2,2,1)
plt.title("May", y=1.08)
mp.imshow(May_jn-May, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,2)
plt.title("June", y=1.08)
mp.imshow(June_jn-June, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,3)
plt.title("July", y=1.08)
mp.imshow(July_jn-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,4)
plt.title("August", y=1.08)
mp.imshow(August_jn-August, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.subplots_adjust(wspace=0.,hspace=0.4, left=0.2, right=0.8)
plt.show()

In [47]:
# August-c1 dominated years
ticks = np.round(np.linspace(-0.25,0.25,10),3)
color = plt.cm.get_cmap('seismic', 9)

plt.figure()
plt.suptitle("Snowcover in May, June, July and August relative to average snowcover(1979-2018)(C1-August Dominated)")

mp=draw_map4()
plt.subplot(2,2,1)
plt.title("May", y=1.08)
mp.imshow(May_ac1-May, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,2)
plt.title("June", y=1.08)
mp.imshow(June_ac1-June, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,3)
plt.title("July", y=1.08)
mp.imshow(July_ac1-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,4)
plt.title("August", y=1.08)
mp.imshow(August_ac1-August, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.subplots_adjust(wspace=0.,hspace=0.4, left=0.2, right=0.8)
plt.show()

In [48]:
# August-c3 dominated years
ticks = np.round(np.linspace(-0.3,0.3,10),3)
color = plt.cm.get_cmap('seismic', 9)
plt.figure()
plt.suptitle("Snowcover in May, June, July and August relative to average snowcover(1979-2018)(C3-August Dominated)")

mp=draw_map4()
plt.subplot(2,2,1)
plt.title("May", y=1.08)
mp.imshow(May_ac3-May, vmax = 0.3, vmin = -0.3, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,2)
plt.title("June", y=1.08)
mp.imshow(June_ac3-June, vmax = 0.3, vmin = -0.3, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,3)
plt.title("July", y=1.08)
mp.imshow(July_ac3-July, vmax = 0.3, vmin = -0.3, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,4)
plt.title("August", y=1.08)
mp.imshow(August_ac3-August, vmax = 0.3, vmin = -0.3, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.subplots_adjust(wspace=0.,hspace=0.4, left=0.2, right=0.8)
plt.show()

In [49]:
# August-neither dominated years
ticks = np.round(np.linspace(-0.25,0.25,10),3)
color = plt.cm.get_cmap('seismic', 9)

plt.figure()
plt.suptitle("Snowcover in May, June, July and August relative to average snowcover(1979-2018)(Neither-August Dominated)")

mp=draw_map4()
plt.subplot(2,2,1)
plt.title("May", y=1.08)
mp.imshow(May_an-May, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,2)
plt.title("June", y=1.08)
mp.imshow(June_an-June, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,3)
plt.title("July", y=1.08)
mp.imshow(July_an-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

#plt.figure()
mp=draw_map4()
plt.subplot(2,2,4)
plt.title("August", y=1.08)
mp.imshow(August_an-August, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.subplots_adjust(wspace=0.,hspace=0.4, left=0.2, right=0.8)
plt.show()

In [50]:
ticks = np.round(np.linspace(-0.25,0.25,10),3)
color = plt.cm.get_cmap('seismic', 9)

plt.figure()
mp=draw_map4()
#plt.subplot(2,2,3)
plt.title("Snowcover in July relative to average snowcover(1979-2018)(C1-July Dominated)", y=1.08)
mp.imshow(July_jc1-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
#plt.subplot(2,2,3)
plt.title("Snowcover in July relative to average snowcover(1979-2018)(C3-July Dominated)", y=1.08)
mp.imshow(July_jc3-July, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
#plt.subplot(2,2,4)
plt.title("Snowcover in August relative to average snowcover(1979-2018)(C1-August Dominated)", y=1.08)
mp.imshow(August_ac1-August, vmax = 0.25, vmin = -0.25, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.figure()
mp=draw_map4()
#plt.subplot(2,2,4)
plt.title("Snowcover in August relative to average snowcover(1979-2018)(C3-August Dominated)", y=1.08)
mp.imshow(August_ac3-August, vmax = 0.3, vmin = -0.3, origin = 'lower', cmap = color,)
mp.colorbar(pad=0.6, ticks = ticks)
mp.drawlsmask(land_color='gray',ocean_color='white', lakes=True, alpha=0.3)
mp.drawparallels(parallels, labels = [0,0,0,0])
mp.drawmeridians(meridians, labels = [1,1,1,1])

plt.show()